In [ ]:
%env CUDA_VISIBLE_DEVICES=""
%env TF_CPP_MIN_LOG_LEVEL="2"

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical



In [ ]:

(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
%matplotlib inline


In [ ]:
image_index = 35
print(y_train[image_index])
plt.imshow(x_train[image_index], cmap='Greys')
plt.show()

In [ ]:
print(x_train.shape)
print(x_test.shape)

In [ ]:
print(y_train[:image_index + 1])

In [ ]:
# save input image dimensions
img_rows, img_cols = 28, 28

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)


In [ ]:

x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

In [ ]:
num_classes = 10

y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [ ]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D


In [ ]:

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
     activation='relu',
     input_shape=(img_rows, img_cols, 1)))

In [ ]:
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [ ]:
model.add(Dropout(0.25))

In [ ]:
model.add(Flatten())

In [ ]:
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
print(model.summary())

In [ ]:
from tensorflow.keras.utils import plot_model


In [ ]:
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
import visualkeras
visualkeras.layered_view(model)

In [ ]:
visualkeras.layered_view(model, legend=True) # without custom font
from PIL import ImageFont
font = ImageFont.truetype("arial.ttf", 12)
visualkeras.layered_view(model, legend=True, font=font) # selected font

In [ ]:
visualkeras.layered_view(model, legend=True, font=font, draw_volume=False)


In [ ]:
model.compile(loss='categorical_crossentropy',
      optimizer='adam',
      metrics=['accuracy'])

In [ ]:
batch_size = 128
epochs = 10

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
model.save("test_model.h5")

In [ ]:
import imageio
#import numpy as np
#from matplotlib import pyplot as plt

im = imageio.imread("test4.jpg")
im = imageio.imread("https://i.imgur.com/a3Rql9C.png")

In [ ]:
from skimage.transform import resize

im = resize(im, (28, 28))
plt.imshow(im)
plt.show()

In [ ]:
gray = np.dot(im[...,:3], [0.299, 0.587, 0.114])
plt.imshow(gray, cmap = plt.get_cmap('gray'))
plt.show()

In [ ]:
# reshape the image
gray = gray.reshape(1, img_rows, img_cols, 1)

# normalize image
# gray /= 255
# print(gray)
# print(img_cols, img_rows)

In [ ]:
# load the model
from tensorflow.keras.models import load_model
model = load_model("test_model.h5")

# predict digit
prediction = model.predict(gray)
print(prediction.argmax())